In [18]:
# increase alpha depend on differential value

import torch
import numpy as np
import output
import importlib
import grad_aco
importlib.reload(output)

# 初期条件
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n = 100
h = 0.0001
alpha = 0.0
alpha_inc = [0.01, 0.005, 0.001]
tau = torch.tensor(1000, device=device)
max_iter = 1000000
lr = [0.001, 0.01, 0.1]
tol = [1e-4, 1e-5, 1e-6]
seed = 42


for a_inc in alpha_inc:
    for lerning_rate in lr:
        for tolerance in tol:
            final_list = []
            best_list = []
            alpha_cnt_list = []
            for i in range(5):
                m = torch.zeros(n, device=device)
                J = grad_aco.initialize_random_parameters(n, seed).to(device)
                alpha = torch.tensor(0.0, device=device)

                # 勾配降下法を実行
                final_energy, best_energy, alpha_cnt = grad_aco.gradient_descent(m, h, J, tau, alpha, a_inc, max_iter, lerning_rate, tolerance)

                # リストに追加
                final_list.append(final_energy.cpu().item())
                best_list.append(best_energy.cpu().item())
                alpha_cnt_list.append(alpha_cnt)

                # modify seed
                seed += 1
            
            # fix seed
            seed = 42

            #save to csv file
            matrix = [[f, b, a] for f, b, a in zip(final_list, best_list, alpha_cnt_list)]
            matrix.insert(0, ['final', 'best', 'alpha'])
            result = output.matrix_to_csv(f'alpha{a_inc}_lr{lerning_rate}_tol{tolerance}_seed{seed}', matrix)

#matrix = [[f, b, a] for f, b, a in zip(final_list, best_list, alpha_cnt_list)]
#matrix.insert(0, ['final', 'best', 'alpha'])
#result = output.matrix_to_csv(f'alpha{a_inc}_lr{lerning_rate}_tol{tolerance}_seed{seed}', matrix)

In [5]:
# increase alpha depend on the schedule

import torch
import numpy as np

# mの正負に応じてイジングモデルのエネルギーを計算
# Not devided by N-1
def energy(m, h, J):
    m = 2 * (m > 0.0) - 1
    return -torch.sum(h * m) - (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m))))

# エネルギー計算関数 (ベクトル化)
# Not devided by N-1
def f(m, h, J, tau, alpha):
    AS = -(1 - alpha) * torch.sum(torch.log(1 - m**2))
    interaction = (torch.sum(J * torch.outer(m, m)) - torch.sum(torch.diagonal(J * torch.outer(m, m))))
    ising = -alpha * (torch.sum(h * m) + interaction)
    return AS + ising

# 勾配計算関数 (ベクトル化)
# Not devided by N-1
def update(m, h, J, tau, alpha):
    interaction_grad = -alpha * (torch.sum(J * m, dim=1) - torch.diagonal(J) * m)
    dif = interaction_grad + (1 - alpha) * m / (1 - m**2) - alpha * h
    return dif

# 勾配降下法 (改良版)
def gradient_descent(m, h, J, tau, alpha, max_iter, tol=1e-6):
    learning_rate = 0.01
    for iteration in range(max_iter):
        grad = update(m, h, J, tau, alpha)
        m = m - learning_rate * grad  # 勾配降下ステップ

        alpha = min(alpha + (1 / max_iter), 0.999999)

    print(alpha)
    return energy(m, h, J)

# シード値の設定
def initialize_random_parameters(n, seed):
    torch.manual_seed(seed)
    J = torch.normal(mean=0.0, std=1.0, size=(n, n))
    return J

# 初期条件
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seed = 42
n = 100
h = 0.0001
tau = torch.tensor(1000, device=device)
max_iter = 1000000
final_list = []

for i in range(1):
    m = torch.zeros(n, device=device)
    J = initialize_random_parameters(n, seed).to(device)
    alpha = torch.tensor(0.0, device=device)

    # 勾配降下法を実行
    final_energy = gradient_descent(m, h, J, tau, alpha, max_iter).cpu().item()

    # リストに追加
    final_list.append(final_energy)

    # modify seed value
    seed += 1

# 最終結果を出力
print("Final mean energy:", np.mean(final_list))


0.999999
Final mean energy: -65.22675323486328
